<a href="https://colab.research.google.com/github/Woojin-02/JAVA-PROGRIMING2/blob/main/2_%EB%AC%B8%EC%9D%98_%EB%82%B4%EC%9A%A9_%EC%9C%A0%ED%98%95_%EB%B6%84%EB%A5%98_%EB%AA%A8%EB%8D%B8%EB%A7%81_%EC%97%90%EC%9D%B4%EB%B8%94%EB%9F%AC_jbk_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **미니프로젝트 4차 1대1 문의 내용 유형 분류기**
# 단계3 : Text classification

### 문제 정의
> 1:1 문의 내용 분류 문제<br>
> 1. 문의 내용 분석
> 2. 문의 내용 분류 모델 성능 평가
### 학습 데이터
> * 1:1 문의 내용 데이터 : train.csv

### 변수 소개
> * text : 문의 내용
> * label : 문의 유형

### References
> * Machine Learning
>> * [sklearn-tutorial](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)
> * Deep Learning
>> * [Google Tutorial](https://developers.google.com/machine-learning/guides/text-classification)
>> * [Tensorflow Tutorial](https://www.tensorflow.org/tutorials/keras/text_classification)
>> * [Keras-tutorial](https://keras.io/examples/nlp/text_classification_from_scratch/)
>> * [BERT-tutorial](https://www.tensorflow.org/text/guide/bert_preprocessing_guide)

## 1. 개발 환경 설정

### 1-1. 라이브러리 설치

In [1]:
# 필요 라이브러리부터 설치할께요.
!pip install konlpy pandas seaborn wordcloud python-mecab-ko wget transformers

### 1-2. 라이브러리 import

In [2]:
from mecab import MeCab
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import wget,os
from IPython.display import display
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import tensorflow as tf
import nltk
import wget,os
from sklearn.metrics import accuracy_score, classification_report

In [3]:
# 런타임 재시작 필요
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/root/.local/share/fonts: skipping, no such directory
/root/.fonts: skipping, no such directory
/usr/share/fonts/truetype: skipping, looped directory detected
/usr/share/fonts/truetype/humor-sans: skipping, looped directory detected
/usr/share/fonts/truetype/liberation: skipping, looped directory detected
/usr/share/fonts/truetype/

### 1-3. 한글 글꼴 설정

In [4]:
!sudo apt-get install -y fonts-nanum

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


In [5]:
FONT_PATH = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = fm.FontProperties(fname=FONT_PATH, size=10).get_name()
print(font_name)
plt.rcParams['font.family']=font_name
assert plt.rcParams['font.family'] == [font_name], "한글 폰트가 설정되지 않았습니다."

NanumGothic


### 1-4. 구글드라이브 연결

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 2. 전처리한 데이터 불러오기
* 1, 2일차에 전처리한 데이터를 불러옵니다.
* sparse data에 대해서는 scipy.sparse.load_npz 활용

In [63]:
# 파일 경로 지정
PATH = '/content/drive/MyDrive/2023.10.16_미니프로젝트4차_실습자료/'

# npy 파일 불러오기
x_mor_sequence_train = np.load("/content/drive/MyDrive/2023.10.16_미니프로젝트4차_실습자료/X_mor_sequence_train.npy")
x_mor_sequence_val = np.load("/content/drive/MyDrive/2023.10.16_미니프로젝트4차_실습자료/X_mor_sequence_val.npy")

x_pr_train = np.load("/content/drive/MyDrive/2023.10.16_미니프로젝트4차_실습자료/X_pr_train.npy")
x_pr_val = np.load("/content/drive/MyDrive/2023.10.16_미니프로젝트4차_실습자료/X_pr_val.npy")

x_w2v_train = np.load("/content/drive/MyDrive/2023.10.16_미니프로젝트4차_실습자료/X_w2v_train.npy")
x_w2v_val = np.load("/content/drive/MyDrive/2023.10.16_미니프로젝트4차_실습자료/X_w2v_val.npy")

y_train = np.load("/content/drive/MyDrive/2023.10.16_미니프로젝트4차_실습자료/y_train.npy")
y_val = np.load("/content/drive/MyDrive/2023.10.16_미니프로젝트4차_실습자료/y_val.npy")

x_mor_sequence_test = np.load("/content/drive/MyDrive/2023.10.16_미니프로젝트4차_실습자료/X_mor_sequence_te.npy")
x_pr_test = np.load("/content/drive/MyDrive/2023.10.16_미니프로젝트4차_실습자료/X_pr_te.npy")
x_w2v_test = np.load("/content/drive/MyDrive/2023.10.16_미니프로젝트4차_실습자료/X_w2v_te.npy")

In [64]:
from scipy import sparse

# npz 파일 불러오기
x_tfidf_train = sparse.load_npz("/content/drive/MyDrive/2023.10.16_미니프로젝트4차_실습자료/X_tfidf_train.npz")
x_tfidf_val = sparse.load_npz("/content/drive/MyDrive/2023.10.16_미니프로젝트4차_실습자료/X_tfidf_val.npz")
x_tfidf_test = sparse.load_npz("/content/drive/MyDrive/2023.10.16_미니프로젝트4차_실습자료/X_tfidf_te.npz")

In [9]:
y_test = pd.read_csv('/content/drive/MyDrive/2023.10.16_미니프로젝트4차_실습자료/test.csv')

In [65]:
from sklearn.model_selection import train_test_split

train = pd.read_csv('/content/drive/MyDrive/2023.10.16_미니프로젝트4차_실습자료/train.csv')
label_dict = {
    '코드1': 0,
    '코드2': 0,
    '웹': 1,
    '이론': 2,
    '시스템 운영': 3,
    '원격': 4
}
preprocessed_df = train.replace({'label':label_dict}).copy()
x_train, x_val, _, _ = train_test_split(preprocessed_df.text, preprocessed_df.label, test_size=0.25, random_state=42)

## 3. Machine Learning(N-grams)
* N-gram으로 전처리한 데이터를 이용하여 3개 이상의 Machine Learning 모델 학습 및 성능 분석
> * [sklearn-tutorial](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html)

### 3-1. Model 1 : RandomForest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

# 파라미터 선언
params = {'max_depth':range(1, 31), 'n_estimators':range(1, 31)}

# 모델 생성
model = GridSearchCV(RandomForestClassifier(), params, cv=5)

# 모델 학습
model.fit(x_tfidf_train, y_train)

# 모델 평가
y_pred = model.predict(x_tfidf_val)

# 정확도 출력
accuracy = accuracy_score(y_val, y_pred)
print("정확도:", accuracy)
print(classification_report(y_val, y_pred))

# 결과 확인
print(model.cv_results_['mean_test_score'])  # 수행 정보 중 평균 성능값들 출력
print('최적파라미터:', model.best_params_)    # 최적 파라미터
print('최고성능:', model.best_score_)         # 최고성능

정확도: 0.7367853290183387
              precision    recall  f1-score   support

           0       0.68      0.95      0.80       392
           1       0.79      0.56      0.66       179
           2       0.76      0.49      0.60       195
           3       0.85      0.71      0.77       130
           4       1.00      0.71      0.83        31

    accuracy                           0.74       927
   macro avg       0.82      0.68      0.73       927
weighted avg       0.76      0.74      0.73       927

[0.43109015 0.43325167 0.44765053 0.432529   0.43324843 0.43073174
 0.42965131 0.42929159 0.42929159 0.43001102 0.42929159 0.42929159
 0.42929159 0.42929159 0.42929159 0.42929159 0.42929159 0.42929159
 0.42929159 0.42929159 0.42929159 0.42929159 0.42929159 0.42929159
 0.42929159 0.42929159 0.42929159 0.42929159 0.42929159 0.42929159
 0.43649167 0.43900966 0.43756433 0.43936678 0.44548188 0.43792598
 0.43037073 0.43396915 0.42965195 0.42965195 0.43181088 0.43037073
 0.42965131 0.4328

### 3-2. Model 2 : SGD

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
clf = MultinomialNB().fit(x_tfidf_train, y_train)

In [ ]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

text_clf.fit(x_train, y_train)
predicted = text_clf.predict(x_val)
np.mean(predicted == y_val)

0.5361380798274002

In [ ]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-3, random_state=42,
                          max_iter=5, tol=None)),
    ])

text_clf.fit(x_train, y_train)
predicted = text_clf.predict(x_val)
np.mean(predicted == y_val)

0.7756202804746494

In [ ]:
from sklearn.model_selection import GridSearchCV
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-2, 1e-3),
}

gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)
gs_clf = gs_clf.fit(x_train, y_train)
predicted = gs_clf.predict(x_val)
np.mean(predicted == y_val)

0.7756202804746494

### 3-3. Model 3 : xgboost

In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV

# 파라미터 선언
params = {'max_depth':range(1, 10)}

# 모델 생성
model_xgb = GridSearchCV(XGBClassifier(), params, cv=5)

# 모델 학습
model_xgb.fit(x_tfidf_train, y_train)

# 모델 평가
y_pred = model.predict(x_tfidf_val)

# 정확도 출력
accuracy = accuracy_score(y_val, y_pred)
print("정확도:", accuracy)
print(classification_report(y_val, y_pred))

KeyboardInterrupt: ignored

### 3-4. Model 4 : Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Logistic Regression 모델 생성 및 학습
model = LogisticRegression()
model.fit(x_tfidf_train, y_train)

# 예측
y_pred = model.predict(x_tfidf_val)

# 모델 평가
accuracy = accuracy_score(y_val, y_pred)
# conf_matrix = confusion_matrix(y_val, y_pred)
class_report = classification_report(y_val, y_pred)

print("Accuracy:", accuracy)
# print("Confusion Matrix:\n", conf_matrix)
print("Classification Report:\n", class_report)

Accuracy: 0.8284789644012945
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.93      0.87       392
           1       0.86      0.81      0.83       179
           2       0.80      0.74      0.77       195
           3       0.89      0.78      0.83       130
           4       1.00      0.42      0.59        31

    accuracy                           0.83       927
   macro avg       0.87      0.74      0.78       927
weighted avg       0.83      0.83      0.82       927



In [ ]:
y_pred_logistic = model.predict(x_tfidf_test)

In [ ]:
y_pred_logistic

array([3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 2, 0, 0, 1, 0, 0, 0, 3, 0, 0, 0, 0,
       0, 0, 3, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0,
       0, 3, 0, 0, 3, 0, 0, 2, 0, 0, 4, 4, 0, 2, 2, 0, 2, 0, 0, 0, 0, 3,
       3, 2, 0, 3, 2, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 3, 3, 0, 0, 0, 0, 0,
       0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 3, 0, 0, 0, 0, 0, 3,
       3, 3, 0, 0, 0, 1, 0, 0, 0, 0, 3, 4, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2, 0, 2, 2, 0, 0, 0, 0, 3, 2,
       2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0,
       0, 0, 0, 3, 3, 0, 3, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 3, 2, 2, 0, 0,
       2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 4, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 0, 2, 1, 0,
       0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 1, 1, 0, 0, 4, 3, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 3, 2, 0, 3, 2, 2, 3, 0, 3, 0, 0,

### 3-4. Hyperparameter Tuning(Optional)
* Manual Search, Grid search, Bayesian Optimization, TPE...
> * [grid search tutorial sklearn](https://scikit-learn.org/stable/modules/grid_search.html)
> * [optuna tutorial](https://optuna.org/#code_examples)
> * [ray-tune tutorial](https://docs.ray.io/en/latest/tune/examples/tune-sklearn.html)

## 4. Machine Learning(N-grams, 강사님)

### Model 1 : SGDClassifier with TF-IDF

In [ ]:
x_tfidf_train.shape, y_train.shape

((2779, 9917), (2779,))

In [ ]:
from sklearn.linear_model import SGDClassifier

print("SGDClassifier(Linear SVM) with TF-IDF")
sgdClf = SGDClassifier(loss='hinge')
sgdClf.fit(X = x_tfidf_train, y = y_train)
y_pred = sgdClf.predict(x_tfidf_val)
print(classification_report(y_val, y_pred))

print("*" * 100)
print("Logistic Regression with TF-IDF")
sgdClf = SGDClassifier(loss='log')
sgdClf.fit(X = x_tfidf_train, y= y_train)
y_pred = sgdClf.predict(x_tfidf_val)
print(classification_report(y_val, y_pred))

SGDClassifier(Linear SVM) with TF-IDF
              precision    recall  f1-score   support

           0       0.84      0.90      0.87       392
           1       0.86      0.85      0.86       179
           2       0.77      0.72      0.74       195
           3       0.84      0.82      0.83       130
           4       1.00      0.84      0.91        31

    accuracy                           0.84       927
   macro avg       0.86      0.82      0.84       927
weighted avg       0.84      0.84      0.84       927

****************************************************************************************************
Logistic Regression with TF-IDF
              precision    recall  f1-score   support

           0       0.84      0.92      0.88       392
           1       0.87      0.83      0.85       179
           2       0.79      0.75      0.77       195
           3       0.88      0.82      0.85       130
           4       1.00      0.61      0.76        31

    accuracy  

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:163: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


### Model 2 : SGDClassifier with Word2Vec

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
print("SGDClassifier with Word2Vec")
w2v_sgd = SGDClassifier(loss='hinge')
w2v_sgd.fit(x_w2v_train, y_train)
y_pred = w2v_sgd.predict(x_w2v_val)
print(classification_report(y_val, y_pred))

print("*" * 100)
print("GBM with Word2Vec")
w2v_gd = GradientBoostingClassifier()
w2v_gd.fit(x_w2v_train, y_train)
y_pred = w2v_gd.predict(x_w2v_val)
print(classification_report(y_val, y_pred))

SGDClassifier with Word2Vec
              precision    recall  f1-score   support

           0       0.57      0.96      0.72       392
           1       0.73      0.27      0.39       179
           2       0.60      0.29      0.39       195
           3       0.78      0.51      0.61       130
           4       0.59      0.52      0.55        31

    accuracy                           0.61       927
   macro avg       0.65      0.51      0.53       927
weighted avg       0.64      0.61      0.57       927

****************************************************************************************************
GBM with Word2Vec
              precision    recall  f1-score   support

           0       0.73      0.81      0.77       392
           1       0.58      0.54      0.56       179
           2       0.61      0.53      0.57       195
           3       0.59      0.61      0.60       130
           4       0.32      0.23      0.26        31

    accuracy                          

### Model 3 : SVM, GBM, ML with Word2Vec(pre_trained)

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
print("SGDClassifier with Word2Vec")
w2v_sgd = SGDClassifier(loss='hinge')
w2v_sgd.fit(x_pr_train, y_train)
y_pred = w2v_sgd.predict(x_pr_val)
print(classification_report(y_val, y_pred))

from sklearn.ensemble import GradientBoostingClassifier

print("*" * 100)
print("GBM with Word2Vec")
w2v_gd = GradientBoostingClassifier()
w2v_gd.fit(x_pr_train, y_train)
y_pred = w2v_gd.predict(x_pr_val)
print(classification_report(y_val, y_pred))

SGDClassifier with Word2Vec
              precision    recall  f1-score   support

           0       0.77      0.80      0.79       392
           1       0.65      0.65      0.65       179
           2       0.63      0.74      0.68       195
           3       0.79      0.62      0.69       130
           4       1.00      0.35      0.52        31

    accuracy                           0.72       927
   macro avg       0.77      0.63      0.67       927
weighted avg       0.73      0.72      0.72       927

****************************************************************************************************
GBM with Word2Vec
              precision    recall  f1-score   support

           0       0.73      0.85      0.79       392
           1       0.62      0.58      0.60       179
           2       0.71      0.59      0.65       195
           3       0.66      0.70      0.68       130
           4       1.00      0.19      0.32        31

    accuracy                          

### Model 4 : Naive Bayes with TF-IDF

In [ ]:
from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB, GaussianNB

In [ ]:
print("MultinomialNB with TF-IDF")
multi = MultinomialNB()
multi.fit(x_tfidf_train, y_train)
y_pred = multi.predict(x_tfidf_val)
print(classification_report(y_val, y_pred, zero_division=0))

print("*" * 100)
print("ComplementNB with TF-IDF")
comple = ComplementNB()
comple.fit(x_tfidf_train, y_train)
y_pred = comple.predict(x_tfidf_val)
print(classification_report(y_val, y_pred))

print("*" * 100)
print("BernoulliNB with TF-IDF")
bernoulli = BernoulliNB()
bernoulli.fit(x_tfidf_train, y_train)
y_pred = bernoulli.predict(x_tfidf_val)
print(classification_report(y_val, y_pred))

print("*" * 100)
print("GaussianNB with TF-IDF")
gauss = GaussianNB()
gauss.fit(x_tfidf_train.toarray(), y_train)
y_pred = gauss.predict(x_tfidf_val.toarray())
print(classification_report(y_val, y_pred))

MultinomialNB with TF-IDF
              precision    recall  f1-score   support

           0       0.53      1.00      0.69       392
           1       0.91      0.39      0.54       179
           2       0.96      0.24      0.39       195
           3       0.92      0.45      0.60       130
           4       0.00      0.00      0.00        31

    accuracy                           0.61       927
   macro avg       0.66      0.41      0.44       927
weighted avg       0.73      0.61      0.56       927

****************************************************************************************************
ComplementNB with TF-IDF
              precision    recall  f1-score   support

           0       0.82      0.94      0.88       392
           1       0.91      0.79      0.85       179
           2       0.83      0.71      0.76       195
           3       0.85      0.90      0.88       130
           4       0.94      0.55      0.69        31

    accuracy                     

In [ ]:
from sklearn.neural_network import MLPClassifier

In [ ]:
print("MLPClassifier with TF-IDF")
mlp = MLPClassifier()
mlp.fit(x_tfidf_train, y_train)
y_pred = mlp.predict(x_tfidf_val)
print(classification_report(y_val, y_pred))

MLPClassifier with TF-IDF
              precision    recall  f1-score   support

           0       0.84      0.91      0.87       392
           1       0.85      0.86      0.85       179
           2       0.79      0.73      0.76       195
           3       0.88      0.84      0.86       130
           4       0.94      0.55      0.69        31

    accuracy                           0.84       927
   macro avg       0.86      0.78      0.81       927
weighted avg       0.84      0.84      0.84       927



## 4. Deep Learning(Sequence)
* Sequence로 전처리한 데이터를 이용하여 DNN, 1-D CNN, LSTM 등 3가지 이상의 deep learning 모델 학습 및 성능 분석
> * [Google Tutorial](https://developers.google.com/machine-learning/guides/text-classification)
> * [Tensorflow Tutorial](https://www.tensorflow.org/tutorials/keras/text_classification)
> * [Keras-tutorial](https://keras.io/examples/nlp/text_classification_from_scratch/)

### 4-1. DNN

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.models import Model

In [ ]:
def _get_last_layer_units_and_activation(num_classes):
    """Gets the # units and activation function for the last network layer.

    # Arguments
        num_classes: int, number of classes.

    # Returns
        units, activation values.
    """
    if num_classes == 2:
        activation = 'sigmoid'
        units = 1
    else:
        activation = 'softmax'
        units = num_classes
    return units, activation

In [ ]:
from tensorflow.keras import models
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

def mlp_model(layers, units, dropout_rate, input_shape, num_classes):
    """Creates an instance of a multi-layer perceptron model.

    # Arguments
        layers: int, number of `Dense` layers in the model.
        units: int, output dimension of the layers.
        dropout_rate: float, percentage of input to drop at Dropout layers.
        input_shape: tuple, shape of input to the model.
        num_classes: int, number of output classes.

    # Returns
        An MLP model instance.
    """
    op_units, op_activation = _get_last_layer_units_and_activation(num_classes)
    model = models.Sequential()
    model.add(Dropout(rate=dropout_rate, input_shape=input_shape))

    for _ in range(layers-1):
        model.add(Dense(units=units, activation='swish'))
        # model.add(BatchNormalization())
        # model.add(Dropout(rate=dropout_rate))

    model.add(Dense(units=op_units, activation=op_activation))
    return model

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# 모델 파라미터 설정
layers = 8  # Dense 레이어 수
units = 1024  # 각 Dense 레이어의 뉴런 수
dropout_rate = 0.0  # Dropout 레이어의 dropout 비율
input_shape = (x_mor_sequence_train.shape[1],)  # 입력 데이터의 형태
num_classes = 5  # 출력 클래스의 수

# MLP 모델 생성
model = mlp_model(layers, units, dropout_rate, input_shape, num_classes)

# 모델 컴파일
model.compile(
    optimizer=Adam(learning_rate=0.001),  # 옵티마이저 설정 (Adam 사용)
    loss='sparse_categorical_crossentropy',  # 손실 함수
    metrics=['accuracy']  # 평가 지표 (정확도)
)

# 모델 요약 정보 출력
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_2 (Dropout)         (None, 500)               0         
                                                                 
 dense_16 (Dense)            (None, 1024)              513024    
                                                                 
 dense_17 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_18 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_19 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_20 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_21 (Dense)            (None, 1024)             

In [ ]:
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, verbose=1, restore_best_weights=True)
history = model.fit(x_mor_sequence_train, y_train, verbose=1, validation_split=0.1, epochs=50) # callbacks=es

Epoch 1/50
79/79 [==============================] - 4s 8ms/step - loss: 16.6918 - accuracy: 0.3543 - val_loss: 1.4906 - val_accuracy: 0.3993
Epoch 2/50
79/79 [==============================] - 0s 5ms/step - loss: 1.3600 - accuracy: 0.4194 - val_loss: 1.5223 - val_accuracy: 0.3633
Epoch 3/50
79/79 [==============================] - 0s 5ms/step - loss: 1.2397 - accuracy: 0.4730 - val_loss: 1.6109 - val_accuracy: 0.2770
Epoch 4/50
79/79 [==============================] - 0s 5ms/step - loss: 1.1822 - accuracy: 0.4986 - val_loss: 1.5729 - val_accuracy: 0.2770
Epoch 5/50
79/79 [==============================] - 0s 5ms/step - loss: 1.0087 - accuracy: 0.5954 - val_loss: 1.6224 - val_accuracy: 0.3237
Epoch 6/50
79/79 [==============================] - 0s 5ms/step - loss: 0.8956 - accuracy: 0.6413 - val_loss: 2.3341 - val_accuracy: 0.3237
Epoch 7/50
79/79 [==============================] - 0s 5ms/step - loss: 0.9176 - accuracy: 0.6709 - val_loss: 2.1566 - val_accuracy: 0.2338
Epoch 8/50
79/79 [=

In [ ]:
model.evaluate(x_mor_sequence_val, y_val)

29/29 [==============================] - 0s 3ms/step - loss: 8.3964 - accuracy: 0.3916


[8.396431922912598, 0.3915857672691345]

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# 모델 파라미터 설정
layers = 8  # Dense 레이어 수
units = 1024  # 각 Dense 레이어의 뉴런 수
dropout_rate = 0.0  # Dropout 레이어의 dropout 비율
input_shape = (x_pr_train.shape[1],)  # 입력 데이터의 형태
num_classes = 5  # 출력 클래스의 수

# MLP 모델 생성
model = mlp_model(layers, units, dropout_rate, input_shape, num_classes)

# 모델 컴파일
model.compile(
    optimizer=Adam(learning_rate=0.001),  # 옵티마이저 설정 (Adam 사용)
    loss='sparse_categorical_crossentropy',  # 손실 함수
    metrics=['accuracy']  # 평가 지표 (정확도)
)

# 모델 요약 정보 출력
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_3 (Dropout)         (None, 100)               0         
                                                                 
 dense_24 (Dense)            (None, 1024)              103424    
                                                                 
 dense_25 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_26 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_27 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_28 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_29 (Dense)            (None, 1024)             

In [ ]:
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, verbose=1, restore_best_weights=True)
history = model.fit(x_pr_train, y_train, verbose=1, callbacks=es, validation_split=0.1, epochs=100)

Epoch 1/100
79/79 [==============================] - 1s 6ms/step - loss: 0.6893 - accuracy: 0.7337 - val_loss: 1.0521 - val_accuracy: 0.5971
Epoch 2/100
79/79 [==============================] - 0s 5ms/step - loss: 0.7025 - accuracy: 0.7285 - val_loss: 0.7835 - val_accuracy: 0.6727
Epoch 3/100
79/79 [==============================] - 0s 5ms/step - loss: 0.6812 - accuracy: 0.7329 - val_loss: 0.7964 - val_accuracy: 0.6942
Epoch 4/100
79/79 [==============================] - 0s 5ms/step - loss: 0.6862 - accuracy: 0.7305 - val_loss: 0.8083 - val_accuracy: 0.7158
Epoch 5/100
79/79 [==============================] - 0s 5ms/step - loss: 0.6345 - accuracy: 0.7597 - val_loss: 0.8473 - val_accuracy: 0.6978
Epoch 6/100
79/79 [==============================] - 0s 5ms/step - loss: 0.6597 - accuracy: 0.7521 - val_loss: 0.8017 - val_accuracy: 0.6871
Epoch 7/100
79/79 [==============================] - 0s 5ms/step - loss: 0.6238 - accuracy: 0.7593 - val_loss: 0.8045 - val_accuracy: 0.7194
Epoch 8/100
7

In [ ]:
model.evaluate(x_pr_val, y_val)

29/29 [==============================] - 0s 3ms/step - loss: 0.7110 - accuracy: 0.7422


[0.710953414440155, 0.7421790957450867]

In [ ]:
y_pred

array([[3.8238525e-01, 2.3671661e-01, 2.1358614e-01, 1.6615693e-01,
        1.1550167e-03],
       [9.9999940e-01, 7.8755406e-08, 1.1236010e-07, 3.3641663e-07,
        2.5142522e-19],
       [2.7679157e-01, 1.9682245e-01, 1.8758838e-01, 2.0367938e-01,
        1.3511825e-01],
       ...,
       [2.8777069e-01, 2.0396349e-01, 1.9230278e-01, 2.0617019e-01,
        1.0979281e-01],
       [2.7679157e-01, 1.9682245e-01, 1.8758838e-01, 2.0367938e-01,
        1.3511825e-01],
       [2.9756188e-01, 2.1021362e-01, 1.9605401e-01, 2.0753504e-01,
        8.8635437e-02]], dtype=float32)

In [ ]:
print(classification_report(y_val, y_pred))

              precision    recall  f1-score   support

           0       0.60      0.89      0.71       392
           1       0.00      0.00      0.00       179
           2       0.27      0.47      0.34       195
           3       0.00      0.00      0.00       130
           4       0.00      0.00      0.00        31

    accuracy                           0.47       927
   macro avg       0.17      0.27      0.21       927
weighted avg       0.31      0.47      0.37       927



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### 4-2. 1-D CNN

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization, Conv1D, MaxPool1D
from tensorflow.keras.models import Model

In [ ]:
# shape 확인
x_mor_sequence_train.shape

(2779, 500)

In [ ]:
# # 1. 세션 클리어
# clear_session()

# # 2. 레이어 엮기
# il = Input(shape=[x_mor_sequence_train.shape[1],])

# hl = Conv1D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu')(il)
# hl = Conv1D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu')(hl)
# hl = MaxPool1D(pool_size=(2, 2), strides=(2, 2))(hl)
# hl = BatchNormalization()(hl)
# hl = Dropout(0.2)(hl)

# hl = Conv1D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu')(hl)
# hl = Conv1D(filters=128, kernel_size=3, strides=1, padding='same', activation='relu')(hl)
# hl = MaxPool2D(pool_size=(2, 2), strides=(2, 2))(hl)
# hl = BatchNormalization()(hl)
# hl = Dropout(0.2)(hl)

# hl = Conv1D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu')(hl)
# hl = Conv1D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu')(hl)
# hl = Conv1D(filters=256, kernel_size=3, strides=1, padding='same', activation='relu')(hl)
# hl = MaxPool2D(pool_size=(2, 2), strides=(2, 2))(hl)
# hl = BatchNormalization()(hl)
# hl = Dropout(0.2)(hl)

# hl = Conv1D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu')(hl)
# hl = Conv1D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu')(hl)
# hl = Conv1D(filters=512, kernel_size=3, strides=1, padding='same', activation='relu')(hl)
# hl = MaxPool2D(pool_size=(2, 2), strides=(2, 2))(hl)
# hl = BatchNormalization()(hl)
# hl = Dropout(0.5)(hl)

# hl = Flatten()(hl)

# hl = Dense(1024, activation='relu')(hl)
# hl = BatchNormalization()(hl)
# hl = Dropout(0.3)(hl)

# ol = Dense(5, activation='softmax')(hl)

# # 3. 모델 선언
# model = Model(il, ol)

# # 4. 모델 컴파일
# model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


# model.summary()

In [ ]:
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, restore_best_weights=True)
history = model.fit(x_mor_sequence_train, y_train, verbose=1, callbacks=es, validation_split=0.2, epochs=10)

In [ ]:
model.evaluate(x_mor_sequence_val, y_val)
y_pred = model.predict(x_mor_sequence_val)
accuracy_score(y_val, y_pred)

### 4-2-1 구글


In [ ]:
from tensorflow.keras import models
from tensorflow.keras import initializers
from tensorflow.keras import regularizers

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SeparableConv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import GlobalAveragePooling1D

def sepcnn_model(blocks,
                 filters,
                 kernel_size,
                 embedding_dim,
                 dropout_rate,
                 pool_size,
                 input_shape,
                 num_classes,
                 num_features,
                 use_pretrained_embedding=False,
                 is_embedding_trainable=False,
                 embedding_matrix=None):
    """Creates an instance of a separable CNN model.

    # Arguments
        blocks: int, number of pairs of sepCNN and pooling blocks in the model.
        filters: int, output dimension of the layers.
        kernel_size: int, length of the convolution window.
        embedding_dim: int, dimension of the embedding vectors.
        dropout_rate: float, percentage of input to drop at Dropout layers.
        pool_size: int, factor by which to downscale input at MaxPooling layer.
        input_shape: tuple, shape of input to the model.
        num_classes: int, number of output classes.
        num_features: int, number of words (embedding input dimension).
        use_pretrained_embedding: bool, true if pre-trained embedding is on.
        is_embedding_trainable: bool, true if embedding layer is trainable.
        embedding_matrix: dict, dictionary with embedding coefficients.

    # Returns
        A sepCNN model instance.
    """
    op_units, op_activation = _get_last_layer_units_and_activation(num_classes)
    model = models.Sequential()

    # Add embedding layer. If pre-trained embedding is used add weights to the
    # embeddings layer and set trainable to input is_embedding_trainable flag.
    if use_pretrained_embedding:
        model.add(Embedding(input_dim=num_features,
                            output_dim=embedding_dim,
                            input_length=input_shape[0],
                            weights=[embedding_matrix],
                            trainable=is_embedding_trainable))
    else:
        model.add(Embedding(input_dim=num_features,
                            output_dim=embedding_dim,
                            input_length=input_shape[0]))

    for _ in range(blocks-1):
        model.add(Dropout(rate=dropout_rate))
        model.add(SeparableConv1D(filters=filters,
                                  kernel_size=kernel_size,
                                  activation='relu',
                                  bias_initializer='random_uniform',
                                  depthwise_initializer='random_uniform',
                                  padding='same'))
        model.add(SeparableConv1D(filters=filters,
                                  kernel_size=kernel_size,
                                  activation='relu',
                                  bias_initializer='random_uniform',
                                  depthwise_initializer='random_uniform',
                                  padding='same'))
        model.add(MaxPooling1D(pool_size=pool_size))
        model.add(BatchNormalization())

    model.add(SeparableConv1D(filters=filters * 2,
                              kernel_size=kernel_size,
                              activation='relu',
                              bias_initializer='random_uniform',
                              depthwise_initializer='random_uniform',
                              padding='same'))
    model.add(SeparableConv1D(filters=filters * 2,
                              kernel_size=kernel_size,
                              activation='relu',
                              bias_initializer='random_uniform',
                              depthwise_initializer='random_uniform',
                              padding='same'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(rate=dropout_rate))
    model.add(Dense(op_units, activation=op_activation))
    return model

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

max_features = x_mor_sequence_train.shape[1]  # Vocabulary size
max_len = 500  # Maximum sequence length

# Create the SepCNN model
model = sepcnn_model(
    blocks=3,
    filters=256,
    kernel_size=3,
    embedding_dim=128,
    dropout_rate=0.4,
    pool_size=3,
    input_shape=(max_len,),
    num_classes=5,
    num_features=max_features,
    use_pretrained_embedding=False,
    is_embedding_trainable=True
)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

NameError: ignored

In [ ]:
# Train the model
epochs = 80
batch_size = 32
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, verbose=1, restore_best_weights=True)
# history = model.fit(x_mor_sequence_train, y_train, verbose=1, callbacks=es, validation_split=0.1, epochs=epochs)
model.fit(x_mor_sequence_train, y_train, verbose=1, epochs=epochs, batch_size=batch_size, validation_split=0.1)

Epoch 1/80
79/79 [==============================] - 2s 28ms/step - loss: 0.8310 - accuracy: 0.6369 - val_loss: 1.2516 - val_accuracy: 0.5360
Epoch 2/80
79/79 [==============================] - 2s 29ms/step - loss: 0.8357 - accuracy: 0.6421 - val_loss: 1.1051 - val_accuracy: 0.5647
Epoch 3/80
79/79 [==============================] - 2s 29ms/step - loss: 0.8185 - accuracy: 0.6437 - val_loss: 1.1720 - val_accuracy: 0.5504
Epoch 4/80
79/79 [==============================] - 1s 19ms/step - loss: 0.8053 - accuracy: 0.6549 - val_loss: 1.2536 - val_accuracy: 0.5468
Epoch 5/80
79/79 [==============================] - 2s 22ms/step - loss: 0.7924 - accuracy: 0.6597 - val_loss: 1.5322 - val_accuracy: 0.4496
Epoch 6/80
79/79 [==============================] - 2s 25ms/step - loss: 0.7821 - accuracy: 0.6573 - val_loss: 1.9151 - val_accuracy: 0.4173
Epoch 7/80
79/79 [==============================] - 1s 19ms/step - loss: 0.7654 - accuracy: 0.6713 - val_loss: 1.8453 - val_accuracy: 0.4353
Epoch 8/80
79

In [ ]:
# Evaluate the model
y_pred = model.predict(x_mor_sequence_val)
y_pred = tf.argmax(y_pred, axis=1).numpy()
# y_val = tf.argmax(y_val, axis=1).numpy()
report = classification_report(y_val, y_pred)
model.evaluate(x_mor_sequence_val, y_val)
print(report)
print("accaurcy :", accuracy_score(y_val, y_pred))

17/29 [================>.............] - ETA: 0s - loss: 1.0351 - accuracy: 0.5717

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


29/29 [==============================] - 0s 7ms/step - loss: 1.0500 - accuracy: 0.5566
              precision    recall  f1-score   support

           0       0.69      0.85      0.77       392
           1       0.33      0.39      0.36       179
           2       0.45      0.10      0.16       195
           3       0.48      0.72      0.58       130
           4       0.00      0.00      0.00        31

    accuracy                           0.56       927
   macro avg       0.39      0.41      0.37       927
weighted avg       0.52      0.56      0.51       927

accaurcy : 0.5566343042071198


### 4-3. LSTM

In [66]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SimpleRNN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# LSTM 모델 정의
model = Sequential()
model.add(Embedding(input_dim=x_mor_sequence_train.shape[1], output_dim=16, input_length=x_mor_sequence_train.shape[1]))
model.add(LSTM(64, return_sequences=True))  # LSTM 레이어 추가
model.add(SimpleRNN(32, return_sequences=True))
model.add(Dense(5, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 500, 16)           8000      
                                                                 
 lstm_2 (LSTM)               (None, 500, 64)           20736     
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 500, 32)           3104      
                                                                 
 dense_76 (Dense)            (None, 500, 5)            165       
                                                                 
Total params: 32005 (125.02 KB)
Trainable params: 32005 (125.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [62]:
# 모델 학습
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, restore_best_weights=True)
model.fit(x_mor_sequence_train, y_train, epochs=10, validation_split=0.1, callbacks=es)

Epoch 1/10


InvalidArgumentError: ignored

In [ ]:
# 모델 평가
loss, accuracy = model.evaluate(x_mor_sequence_val, y_val)
print(f"val loss: {loss:.4f}, val accuracy: {accuracy:.4f}")

report = classification_report(y_val, y_pred)
print(report)
print("accaurcy :", accuracy_score(y_val, y_pred))

## 5. Using pre-trained model(Optional)
* 한국어 pre-trained model로 fine tuning 및 성능 분석
> * [BERT-tutorial](https://www.tensorflow.org/text/guide/bert_preprocessing_guide)
> * [HuggingFace-Korean](https://huggingface.co/models?language=korean)

## Kaggle

### deeplearning

In [11]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.backend import clear_session
from tensorflow.keras.layers import Input, Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.models import Model

In [12]:
def _get_last_layer_units_and_activation(num_classes):
    """Gets the # units and activation function for the last network layer.

    # Arguments
        num_classes: int, number of classes.

    # Returns
        units, activation values.
    """
    if num_classes == 2:
        activation = 'sigmoid'
        units = 1
    else:
        activation = 'softmax'
        units = num_classes
    return units, activation

In [41]:
from tensorflow.keras import models
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

def mlp_model(layers, units, dropout_rate, input_shape, num_classes):
    """Creates an instance of a multi-layer perceptron model.

    # Arguments
        layers: int, number of `Dense` layers in the model.
        units: int, output dimension of the layers.
        dropout_rate: float, percentage of input to drop at Dropout layers.
        input_shape: tuple, shape of input to the model.
        num_classes: int, number of output classes.

    # Returns
        An MLP model instance.
    """
    op_units, op_activation = _get_last_layer_units_and_activation(num_classes)
    model = models.Sequential()
    model.add(Dropout(rate=dropout_rate, input_shape=input_shape))

    for _ in range(layers-1):
        model.add(Dense(units=units, activation='swish'))
        # model.add(BatchNormalization())
        # model.add(Dropout(rate=dropout_rate))

    model.add(Dense(units=op_units, activation=op_activation))
    return model

In [42]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# 모델 파라미터 설정
layers = 8  # Dense 레이어 수
units = 1024  # 각 Dense 레이어의 뉴런 수
dropout_rate = 0.1  # Dropout 레이어의 dropout 비율
input_shape = (x_pr_train.shape[1],)  # 입력 데이터의 형태
num_classes = 5  # 출력 클래스의 수

# MLP 모델 생성
model = mlp_model(layers, units, dropout_rate, input_shape, num_classes)

# 모델 컴파일
model.compile(
    optimizer=Adam(learning_rate=0.001),  # 옵티마이저 설정 (Adam 사용)
    loss='sparse_categorical_crossentropy',  # 손실 함수
    metrics=['accuracy']  # 평가 지표 (정확도)
)

# 모델 요약 정보 출력
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dropout_17 (Dropout)        (None, 100)               0         
                                                                 
 dense_52 (Dense)            (None, 1024)              103424    
                                                                 
 dense_53 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_54 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_55 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_56 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_57 (Dense)            (None, 1024)             

In [39]:
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=50, verbose=1, restore_best_weights=True)
history = model.fit(x_pr_train, y_train, verbose=1, callbacks=es, validation_split=0.1, epochs=100)

Epoch 1/100
79/79 [==============================] - 9s 23ms/step - loss: 1.8971 - accuracy: 0.4986 - val_loss: 1.6208 - val_accuracy: 0.3705
Epoch 2/100
79/79 [==============================] - 1s 17ms/step - loss: 1.0431 - accuracy: 0.6074 - val_loss: 1.5427 - val_accuracy: 0.3705
Epoch 3/100
79/79 [==============================] - 1s 16ms/step - loss: 0.9612 - accuracy: 0.6277 - val_loss: 1.8066 - val_accuracy: 0.3705
Epoch 4/100
79/79 [==============================] - 1s 12ms/step - loss: 0.9297 - accuracy: 0.6465 - val_loss: 1.9428 - val_accuracy: 0.3705
Epoch 5/100
79/79 [==============================] - 1s 12ms/step - loss: 0.9368 - accuracy: 0.6397 - val_loss: 1.3729 - val_accuracy: 0.4137
Epoch 6/100
79/79 [==============================] - 1s 12ms/step - loss: 0.9157 - accuracy: 0.6377 - val_loss: 1.3088 - val_accuracy: 0.4245
Epoch 7/100
79/79 [==============================] - 1s 12ms/step - loss: 0.8940 - accuracy: 0.6533 - val_loss: 1.1339 - val_accuracy: 0.5576
Epoch 

In [40]:
model.evaluate(x_pr_val, y_val)

29/29 [==============================] - 0s 3ms/step - loss: 0.7791 - accuracy: 0.6926


[0.7790585160255432, 0.692556619644165]

### deeplearning skip conection

In [51]:
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Dropout, Add, Input
from tensorflow.keras.models import Model

def mlp_model(layers, units, dropout_rate, input_shape, num_classes):
    """Creates an instance of a multi-layer perceptron model with skip connections.

    # Arguments
        layers: int, number of `Dense` layers in the model.
        units: int, output dimension of the layers.
        dropout_rate: float, percentage of input to drop at Dropout layers.
        input_shape: tuple, shape of input to the model.
        num_classes: int, number of output classes.

    # Returns
        An MLP model instance with skip connections.
    """
    op_units, op_activation = _get_last_layer_units_and_activation(num_classes)

    inputs = Input(shape=input_shape)
    x = Dropout(rate=dropout_rate)(inputs)

    for _ in range(layers - 1):
        x = Dense(units=units, activation='swish')(x)
        x = Add()([x, inputs])  # Add a skip connection here
        x = Dropout(rate=dropout_rate)(x)

    outputs = Dense(units=op_units, activation=op_activation)(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
def mlp_model(layers, units, dropout_rate, input_shape, num_classes):
    op_units, op_activation = _get_last_layer_units_and_activation(num_classes)
    model = models.Sequential()
    model.add(Dropout(rate=dropout_rate, input_shape=input_shape))

    for _ in range(layers-1):
        model.add(Dense(units=units, activation='swish'))
        # model.add(BatchNormalization())
        # model.add(Dropout(rate=dropout_rate))

    model.add(Dense(units=op_units, activation=op_activation))
    return model

In [45]:
x_pr_train.shape[1]

100

In [54]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# 모델 파라미터 설정
layers = 8  # Dense 레이어 수
units = 512  # 각 Dense 레이어의 뉴런 수
dropout_rate = 0.0  # Dropout 레이어의 dropout 비율
input_shape = (x_pr_train.shape[1],)  # 입력 데이터의 형태
num_classes = 5  # 출력 클래스의 수

print(input_shape)

# MLP 모델 생성
model = mlp_model(layers, units, dropout_rate, input_shape, num_classes)

# 모델 컴파일
model.compile(
    optimizer=Adam(learning_rate=0.001),  # 옵티마이저 설정 (Adam 사용)
    loss='sparse_categorical_crossentropy',  # 손실 함수
    metrics=['accuracy']  # 평가 지표 (정확도)
)

# 모델 요약 정보 출력
model.summary()

(100,)


ValueError: ignored

In [48]:
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=50, verbose=1, restore_best_weights=True)
history = model.fit(x_pr_train, y_train, verbose=1, callbacks=es, validation_split=0.1, epochs=100)

Epoch 1/100


ValueError: ignored

In [ ]:
model.evaluate(x_pr_val, y_val)

### CNN

In [55]:
from tensorflow.keras import models
from tensorflow.keras import initializers
from tensorflow.keras import regularizers

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import SeparableConv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import GlobalAveragePooling1D

def sepcnn_model(blocks,
                 filters,
                 kernel_size,
                 embedding_dim,
                 dropout_rate,
                 pool_size,
                 input_shape,
                 num_classes,
                 num_features,
                 use_pretrained_embedding=False,
                 is_embedding_trainable=False,
                 embedding_matrix=None):
    """Creates an instance of a separable CNN model.

    # Arguments
        blocks: int, number of pairs of sepCNN and pooling blocks in the model.
        filters: int, output dimension of the layers.
        kernel_size: int, length of the convolution window.
        embedding_dim: int, dimension of the embedding vectors.
        dropout_rate: float, percentage of input to drop at Dropout layers.
        pool_size: int, factor by which to downscale input at MaxPooling layer.
        input_shape: tuple, shape of input to the model.
        num_classes: int, number of output classes.
        num_features: int, number of words (embedding input dimension).
        use_pretrained_embedding: bool, true if pre-trained embedding is on.
        is_embedding_trainable: bool, true if embedding layer is trainable.
        embedding_matrix: dict, dictionary with embedding coefficients.

    # Returns
        A sepCNN model instance.
    """
    op_units, op_activation = _get_last_layer_units_and_activation(num_classes)
    model = models.Sequential()

    # Add embedding layer. If pre-trained embedding is used add weights to the
    # embeddings layer and set trainable to input is_embedding_trainable flag.
    if use_pretrained_embedding:
        model.add(Embedding(input_dim=num_features,
                            output_dim=embedding_dim,
                            input_length=input_shape[0],
                            weights=[embedding_matrix],
                            trainable=is_embedding_trainable))
    else:
        model.add(Embedding(input_dim=num_features,
                            output_dim=embedding_dim,
                            input_length=input_shape[0]))

    for _ in range(blocks-1):
        model.add(Dropout(rate=dropout_rate))
        model.add(SeparableConv1D(filters=filters,
                                  kernel_size=kernel_size,
                                  activation='relu',
                                  bias_initializer='random_uniform',
                                  depthwise_initializer='random_uniform',
                                  padding='same'))
        model.add(SeparableConv1D(filters=filters,
                                  kernel_size=kernel_size,
                                  activation='relu',
                                  bias_initializer='random_uniform',
                                  depthwise_initializer='random_uniform',
                                  padding='same'))
        model.add(MaxPooling1D(pool_size=pool_size))
        model.add(BatchNormalization())

    model.add(SeparableConv1D(filters=filters * 2,
                              kernel_size=kernel_size,
                              activation='relu',
                              bias_initializer='random_uniform',
                              depthwise_initializer='random_uniform',
                              padding='same'))
    model.add(SeparableConv1D(filters=filters * 2,
                              kernel_size=kernel_size,
                              activation='relu',
                              bias_initializer='random_uniform',
                              depthwise_initializer='random_uniform',
                              padding='same'))
    model.add(GlobalAveragePooling1D())
    model.add(Dropout(rate=dropout_rate))
    model.add(Dense(op_units, activation=op_activation))
    return model

In [56]:
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

max_features = x_mor_sequence_train.shape[1]  # Vocabulary size
max_len = 500  # Maximum sequence length

# Create the SepCNN model
model = sepcnn_model(
    blocks=4,
    filters=256,
    kernel_size=3,
    embedding_dim=16,
    dropout_rate=0.1,
    pool_size=3,
    input_shape=(max_len,),
    num_classes=5,
    num_features=max_features,
    use_pretrained_embedding=False,
    is_embedding_trainable=True
)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 16)           8000      
                                                                 
 dropout_31 (Dropout)        (None, 500, 16)           0         
                                                                 
 separable_conv1d (Separabl  (None, 500, 256)          4400      
 eConv1D)                                                        
                                                                 
 separable_conv1d_1 (Separa  (None, 500, 256)          66560     
 bleConv1D)                                                      
                                                                 
 max_pooling1d (MaxPooling1  (None, 166, 256)          0         
 D)                                                              
                                                      

In [57]:
# Train the model
epochs = 20
batch_size = 32
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, verbose=1, restore_best_weights=True)
# history = model.fit(x_mor_sequence_train, y_train, verbose=1, callbacks=es, validation_split=0.1, epochs=epochs)
model.fit(x_mor_sequence_train, y_train, verbose=1, epochs=epochs, batch_size=batch_size, validation_split=0.1)

Epoch 1/80
79/79 [==============================] - 26s 191ms/step - loss: 1.4310 - accuracy: 0.4350 - val_loss: 1.4620 - val_accuracy: 0.3705
Epoch 2/80
79/79 [==============================] - 11s 139ms/step - loss: 1.3801 - accuracy: 0.4358 - val_loss: 1.4333 - val_accuracy: 0.3705
Epoch 3/80
79/79 [==============================] - 10s 123ms/step - loss: 1.3785 - accuracy: 0.4358 - val_loss: 1.4544 - val_accuracy: 0.3705
Epoch 4/80
79/79 [==============================] - 6s 79ms/step - loss: 1.3215 - accuracy: 0.4358 - val_loss: 1.4481 - val_accuracy: 0.3705
Epoch 5/80
79/79 [==============================] - 6s 80ms/step - loss: 1.2063 - accuracy: 0.4482 - val_loss: 1.3917 - val_accuracy: 0.4137
Epoch 6/80
79/79 [==============================] - 4s 51ms/step - loss: 1.0822 - accuracy: 0.5510 - val_loss: 1.2618 - val_accuracy: 0.4676
Epoch 7/80
79/79 [==============================] - 5s 57ms/step - loss: 0.9945 - accuracy: 0.5838 - val_loss: 1.2793 - val_accuracy: 0.3777
Epoch 8

In [58]:
# Evaluate the model
y_pred = model.predict(x_mor_sequence_val)
y_pred = tf.argmax(y_pred, axis=1).numpy()
# y_val = tf.argmax(y_val, axis=1).numpy()
report = classification_report(y_val, y_pred)
model.evaluate(x_mor_sequence_val, y_val)
print(report)
print("accaurcy :", accuracy_score(y_val, y_pred))

29/29 [==============================] - 0s 8ms/step - loss: 2.3431 - accuracy: 0.6440
              precision    recall  f1-score   support

           0       0.58      0.94      0.71       392
           1       0.77      0.43      0.55       179
           2       0.88      0.24      0.37       195
           3       0.73      0.60      0.66       130
           4       0.93      0.90      0.92        31

    accuracy                           0.64       927
   macro avg       0.78      0.62      0.64       927
weighted avg       0.71      0.64      0.61       927

accaurcy : 0.6440129449838188


### LSTM

In [98]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SimpleRNN
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

# LSTM 모델 정의
model = Sequential()
model.add(Embedding(input_dim=x_mor_sequence_train.shape[1], output_dim=16, input_length=x_mor_sequence_train.shape[1]))
model.add(LSTM(64, return_sequences=True))  # LSTM 레이어 추가
model.add(SimpleRNN(32))
# model.add(SimpleRNN(64, return_sequences=True))
# model.add(LSTM(128, return_sequences=True))  # LSTM 레이어 추가
# model.add(SimpleRNN(128))
model.add(Dense(5, activation='softmax'))

# 모델 컴파일
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_15 (Embedding)    (None, 500, 16)           8000      
                                                                 
 lstm_20 (LSTM)              (None, 500, 64)           20736     
                                                                 
 simple_rnn_4 (SimpleRNN)    (None, 128)               24704     
                                                                 
 dense_87 (Dense)            (None, 5)                 645       
                                                                 
Total params: 54085 (211.27 KB)
Trainable params: 54085 (211.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# 모델 학습
es = EarlyStopping(monitor='val_loss', min_delta=0, patience=30, verbose=1, restore_best_weights=True)
model.fit(x_mor_sequence_train, y_train, epochs=20, validation_split=0.1, callbacks=es)

Epoch 1/20
79/79 [==============================] - 55s 663ms/step - loss: 1.4010 - accuracy: 0.4266 - val_loss: 1.3402 - val_accuracy: 0.3669
Epoch 2/20
79/79 [==============================] - 50s 628ms/step - loss: 1.1891 - accuracy: 0.5142 - val_loss: 1.1451 - val_accuracy: 0.5036
Epoch 3/20
79/79 [==============================] - 48s 612ms/step - loss: 1.2681 - accuracy: 0.4902 - val_loss: 1.3823 - val_accuracy: 0.3813
Epoch 4/20
79/79 [==============================] - 48s 614ms/step - loss: 1.2097 - accuracy: 0.4810 - val_loss: 1.2358 - val_accuracy: 0.4137
Epoch 5/20
79/79 [==============================] - 50s 624ms/step - loss: 1.1370 - accuracy: 0.5326 - val_loss: 1.2305 - val_accuracy: 0.4928
Epoch 6/20
79/79 [==============================] - 47s 585ms/step - loss: 1.0929 - accuracy: 0.5330 - val_loss: 1.1603 - val_accuracy: 0.5144
Epoch 7/20
79/79 [==============================] - 45s 571ms/step - loss: 1.0700 - accuracy: 0.5358 - val_loss: 1.2710 - val_accuracy: 0.4820

In [ ]:
# 모델 평가
loss, accuracy = model.evaluate(x_mor_sequence_val, y_val)
print(f"val loss: {loss:.4f}, val accuracy: {accuracy:.4f}")